The Langchain `PanelCallbackHandler` itself is not a widget or pane, but is useful for rendering and streaming output from Langchain Tools, Agents, and Chains as `ChatMessage` objects. It inherits from Langchain's [BaseCallbackHandler](https://python.langchain.com/docs/modules/callbacks/).

#### Parameters:

##### Core

* **`instance`** (ChatFeed | ChatInterface): The ChatFeed or ChatInterface instance.
* **`user`** (str): Name of the user who sent the message.
* **`avatar`** (str | BinaryIO): The avatar to use for the user. Can be a single character text, an emoji, or anything supported by `pn.pane.Image`. If not set, uses the first character of the name.

___

#### Basics

To get started:

1. Pass the instance of a `ChatFeed` or `ChatInterface` to `PanelCallbackHandler`.
2. Pass the `callback_handler` as a list into `callbacks` when constructing Langchain objects.


```python
import panel as pn
from langchain.llms import OpenAI

pn.extension()


def callback(contents, user, instance):
    llm.predict(contents)


instance = pn.chat.ChatInterface(callback=callback)
callback_handler = pn.chat.PanelCallbackHandler(instance)

llm = OpenAI(temperature=0, callbacks=[callback_handler])

instance.servable()
```

#### Async

`async` can also be used:

1. Prefix the function with `async`.
2. Replace the `predict` call with `apredict`.
3. Prefix the call with `await`.

```python
import panel as pn
from langchain.llms import OpenAI

pn.extension()


async def callback(contents, user, instance):
    await llm.apredict(contents)


instance = pn.chat.ChatInterface(callback=callback)
callback_handler = pn.chat.PanelCallbackHandler(instance)

llm = OpenAI(temperature=0, callbacks=[callback_handler])

instance.servable()
```

#### Streaming

To stream tokens from the LLM, simply set `streaming=True` when constructing the LLM.

Note, `async` is not required to stream, but it is more efficient.

```python
import panel as pn
from langchain.llms import OpenAI

pn.extension()


async def callback(contents, user, instance):
    await llm.apredict(contents)


instance = pn.chat.ChatInterface(callback=callback)
callback_handler = pn.chat.PanelCallbackHandler(instance)

llm = OpenAI(temperature=0, callbacks=[callback_handler], streaming=True)

instance.servable()
```

### Agents & Tools

To differentiate between agents', tools', and other LLM output (i.e. automatically use corresponding avatars and names), be sure to also provide `callback_handler` to their constructors.

Again, `async` is not required, but more efficient.

```python
import panel as pn
from langchain.agents import AgentType, load_tools, initialize_agent
from langchain.llms import OpenAI

pn.extension()


async def callback(contents, *args):
    await agent.arun(contents)


instance = pn.chat.ChatInterface(callback=callback)
callback_handler = pn.chat.PanelCallbackHandler(instance)
llm = OpenAI(temperature=0, callbacks=[callback_handler], streaming=True)
tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=[callback_handler])
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=[callback_handler],
)

instance.servable()
```